# Loan Default Prediction
## Team Members:
* Harish Puvvada
* Vamsi Mohan Ramineedi

In [19]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)
import warnings
warnings.filterwarnings('ignore')

In [2]:
#df2007_11 = pd.read_csv(os.getenv('FDS')+'LoanStats_2007_to_2011.csv',low_memory=False,skiprows=1)
df2012_13 = pd.read_csv(os.getenv('FDS')+'LoanStats_2012_to_2013.csv',low_memory=False,skiprows=1)
df2014 = pd.read_csv(os.getenv('FDS')+'LoanStats_2014.csv',low_memory=False,skiprows=1)

In [3]:
dataset = pd.concat([df2012_13, df2014]) #merging 2007 to 2014 datasets
dataset = dataset.iloc[:,2:111] #removing empty columns
empty_cols = [i for i in range(45,72)] #more empty columns
dataset = dataset.drop(dataset.columns[empty_cols],axis=1)
data_with_loanstatus_sliced = dataset[(dataset['loan_status']=="Fully Paid") | (dataset['loan_status']=="Charged Off")]
di = {"Fully Paid":0, "Charged Off":1} #converting target variable to boolean
Dataset_withBoolTarget= data_with_loanstatus_sliced.replace({"loan_status": di})

In [4]:
Dataset_withBoolTarget['loan_status'].value_counts()
Dataset_withBoolTarget.shape

(376233, 82)

In [5]:
#print(Dataset_withBoolTarget.shape)
dataset=Dataset_withBoolTarget.dropna(thresh = 340000,axis=1) #340000 is minimum number of non-NA values
#print(x.isnull().sum()) #- to check how many null values in all the columns
print(dataset.shape)
#dataset['loan_status'].value_counts()

(376233, 74)


In [6]:
del_col_names = ["delinq_2yrs",  "last_pymnt_d", "chargeoff_within_12_mths","delinq_amnt","emp_title", "term", "emp_title", "pymnt_plan","purpose","title", "zip_code", "verification_status", "dti","earliest_cr_line", "initial_list_status", "out_prncp",
"pymnt_plan", "num_tl_90g_dpd_24m", "num_tl_30dpd", "num_tl_120dpd_2m", "num_accts_ever_120_pd", "delinq_amnt", 
"chargeoff_within_12_mths", "total_rec_late_fee", "out_prncp_inv", "issue_d"] #deleting some more columns
dataset = dataset.drop(labels = del_col_names, axis = 1) 
dataset.shape


(376233, 52)

In [7]:
features = ['funded_amnt','emp_length','annual_inc','home_ownership','addr_state','tax_liens','grade',
            "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy",
            "avg_cur_bal","acc_open_past_24mths",'loan_status'] #selecting final features
Final_data = dataset[features] #19 features with target var
Final_data.head()
#Final_data.shape

,funded_amnt,emp_length,annual_inc,home_ownership,addr_state,tax_liens,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
0,27050.0,10+ years,55000.0,OWN,MI,0.0,B,6074.19,4.0,0.0,10.99%,14.0,4.0,16.0,326.0,53.9,16473.0,9570.0,3.0,0
1,12000.0,4 years,60000.0,RENT,NC,0.0,B,3775.55,0.0,0.0,10.99%,15.0,7.0,1.0,48.0,15.9,15216.0,476.0,8.0,0
2,4800.0,2 years,39600.0,MORTGAGE,TX,0.0,B,3900.48,0.0,0.0,10.99%,3.0,2.0,25.0,220.0,16.1,21564.0,1379.0,0.0,0
3,12000.0,10+ years,130000.0,MORTGAGE,CO,0.0,B,398.28,3.0,0.0,11.99%,9.0,5.0,4.0,193.0,93.0,3567.0,36362.0,4.0,0
4,7550.0,3 years,28000.0,RENT,CA,0.0,C,529.67,0.0,0.0,16.24%,4.0,4.0,17.0,38.0,96.0,160.0,1440.0,1.0,0


In [8]:
HTML(Final_data.tail().to_html())

,funded_amnt,emp_length,annual_inc,home_ownership,addr_state,tax_liens,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
235623,15000.0,4 years,66000.0,RENT,MD,0.0,B,509.12,0.0,0.0,13.53%,8.0,6.0,50.0,120.0,96.5,580.0,2877.0,1.0,0
235624,18400.0,4 years,110000.0,MORTGAGE,TX,0.0,C,7394.95,4.0,0.0,14.47%,18.0,7.0,15.0,128.0,89.6,2203.0,17353.0,5.0,0
235625,22000.0,10+ years,78000.0,MORTGAGE,TN,0.0,D,17.50,5.0,1.0,19.97%,18.0,5.0,17.0,198.0,54.4,5569.0,13864.0,5.0,1
235626,2000.0,3 years,83000.0,OWN,CA,0.0,A,1500.68,2.0,0.0,7.90%,21.0,3.0,5.0,130.0,23.4,36402.0,28172.0,7.0,0
235628,10000.0,10+ years,46000.0,MORTGAGE,CA,0.0,D,367.20,2.0,0.0,19.20%,6.0,2.0,4.0,166.0,83.9,2175.0,11495.0,4.0,0


In [27]:
#Final_data["int_rate"]
Final_data["int_rate"] = Final_data["int_rate"].replace({'%':''}, regex = True) #removing % sign from int_rate
Final_data['int_rate'].astype('float64')

0         10.99
1         10.99
2         10.99
3         11.99
4         16.24
5         13.53
6          9.67
7         19.97
8         10.99
9         14.47
10        22.90
11         7.62
12         8.90
13        13.53
14        13.53
15        12.85
16         7.62
17        13.98
18        14.98
19         6.62
20        12.85
21         7.90
22         7.62
23        15.61
24         9.67
26        13.98
27         7.62
28        14.98
30        16.24
31        10.99
          ...  
235597    14.98
235598    23.40
235599    18.25
235600    13.53
235601    16.99
235602     9.67
235603    21.48
235604    15.61
235605    13.53
235606    16.24
235607    13.98
235608    17.57
235609     7.90
235610     8.90
235611    12.85
235612    22.90
235613    16.24
235614    22.40
235615     7.90
235617    10.99
235618    11.99
235619    15.61
235620    18.25
235621     7.90
235622    18.25
235623    13.53
235624    14.47
235625    19.97
235626     7.90
235628    19.20
Name: int_rate, Length: 

In [29]:
Final_data.columns
#dataset["open_acc"].value_counts()
Final_data["int_rate"].corr(Final_data["loan_status"])

KeyboardInterrupt: 

In [ ]:
#dataset[(dataset['avg_cur_bal'] < 5000 ) | (dataset['loan_status']==1)]

In [ ]:
features = []

In [ ]:
mandatory  - "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl","mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy","avg_cur_bal","acc_open_past_24mths"

In [ ]:
doubtful - acc_now_delinq, pct_tl_nvr_dlq, num_sats, sub_grade